# Deep Learning Toolkit for Splunk - Barebone Notebook

This notebook contains a barebone example workflow how to work on custom containerized code that seamlessly interfaces with the Deep Learning Toolkit for Splunk.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import os
import torch
import torch.utils.data
import torch.nn as nn
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

numpy version: 1.15.4
pandas version: 0.25.1


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer mode=stage algo=barebone epochs=10 batch_size=1 s from feature_* into app:barebone_model

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [3]:
# should be the name of directory you created to save your features data
data_dir = 'data'

In [4]:
# take a look at some matsnu example domains
from dga import matsnu

for i in range(10):
    print(matsnu.generate_domain())

fuelquoteprocessdogapproach.com
sidefilmdiscovermortgage.com
benefittoweltransitionboot.com
abilityassociatelacktelephone.com
eyesuggestinfluencefillrepair.com
choicedoubtconsistmarket.com
articlediffergrandfather.com
understandingcreaminfluence.com
holidayexerciseafforddump.com
factorpagerecommendexchange.com


In [5]:
# matsnu domains
matsnu_list = []

for i in range(20000):
    matsnu_list.append(matsnu.generate_domain())
    
matsnu_df = pd.DataFrame(matsnu_list, columns=['domain'])

print("Matsnu Shape:", matsnu_df.shape)

matsnu_df['label'] = 'dga'

matsnu_df.head()

Matsnu Shape: (20000, 1)


,domain,label
0,plastictowelinformresort.com,dga
1,bridgelimitedbrickclaimtax.com,dga
2,featuresuspectplaceselect.com,dga
3,knifeoccurappreciateease.com,dga
4,applespringclaimboarddevil.com,dga


In [6]:
# alex top 1 million domains
alexa_df = pd.read_csv(data_dir + "/alexa_top_1m.csv", names=['domain'])

alexa_df['label'] = 'benign'

alexa_df.head()

,domain,label
1,google.com,benign
2,youtube.com,benign
3,facebook.com,benign
4,baidu.com,benign
5,wikipedia.org,benign


In [7]:
train_df = pd.concat([alexa_df.iloc[:20000], matsnu_df.iloc[:20000]], axis=0).reset_index(drop=True)

In [8]:
train_df.head()

,domain,label
0,google.com,benign
1,youtube.com,benign
2,facebook.com,benign
3,baidu.com,benign
4,wikipedia.org,benign


In [9]:
# import labelencoder
from sklearn.preprocessing import LabelEncoder

# instantiate labelencoder object
le = LabelEncoder()

In [10]:
# Apply le on categorical feature columns
train_df['label'] = le.fit_transform(train_df['label'])

train_df['label'].head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: label, dtype: int64

In [11]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
import numpy as np
from sklearn.preprocessing import LabelEncoder
# from __future__ import unicode_literals, print_function, division
from io import open
import glob

# print(findFiles('data/names/*.txt'))

import unicodedata
import string

class DGADataset(Dataset):
    def __init__(self, data_dir, csv_path, n_records):
        """
        Args:
            data_dir (string): directory name
            csv_path (string): csv filename
        """
        
        self.all_chars =  self.__build__chars__()
        
        self.alexa_df = pd.read_csv(
            os.path.join(data_dir, csv_path),
            names=['domain']
        ).iloc[:n_records]
        self.alexa_df['label'] = 'benign'
        
        self.matsnu_df = pd.DataFrame(
            [matsnu.generate_domain() for i in range(n_records)],
            columns = ['domain']
        )
        self.matsnu_df['label'] = 'dga'
        
        self.data_df = pd.concat([self.alexa_df, self.matsnu_df]).reset_index(drop=True)
        
        class_labeler = LabelEncoder()
        self.data_df['class'] = le.fit_transform(self.data_df['label'])
        
        self.inputs = self.data_df.iloc[:, 0]
        self.labels = self.data_df.iloc[:, -1]
        
        self.data_len = len(self.data_df.index)

    def __build__chars__(self):
        """Build dictionary of chars."""
        all_letters = string.ascii_letters + string.digits + " .'-"
        return {all_letters[i]:i for i in range(len(all_letters))}
    
    def char_to_ix(self, char):
        """Character to index lookup."""
        return self.all_chars[char]

    def ix_to_char(self, char):
        """Index to character lookup."""
        for i, val in self.all_chars.items():
            if val == char:
                return i

    def domain_to_ix(self, domain):
        """Domain to sequence of indexes."""
        return torch.LongTensor([self.char_to_ix(i) for i in domain])

    def __getitem__(self, index):
        domain = self.domain_to_ix(self.inputs[index])
        target = torch.LongTensor([self.labels[index]])
        return (domain, target)

    def __len__(self):
        return self.data_len


In [12]:
DGADataset('data', 'alexa_top_1m.csv', 10).__getitem__(0)

(tensor([ 6, 14, 14,  6, 11,  4, 63,  2, 14, 12]), tensor([0]))

In [13]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def pad_collate(batch):
    (xx, yy) = zip(*batch)
    x_lens = [len(x) for x in xx]
    y_lens = [len(y) for y in yy]

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=0)

    return xx_pad, yy, x_lens, y_lens

def _get_train_data_loader(batch_size, data_dir, filename, n_records):
    print("Get train data loader.")
    
    dataset =  DGADataset(data_dir, filename, n_records)
    
    return DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=pad_collate)

In [14]:
import unittest

class TestDGADataSet(unittest.TestCase):
    def setUp(self):
        self.n = 200
        self.test_dga = DGADataset('data', 'alexa_top_1m.csv', self.n)

    def test_length(self):
        actual_len = self.test_dga.__len__()
        self.assertEqual(actual_len, self.n * 2, 
                         "DataSet is not the right length ({}), should be {}".format(actual_len, 400)
                        )

    def test_labels(self):
        expected_labels = ['benign', 'dga']
        labels = self.test_dga.data_df['label'].unique().tolist()
        self.assertEqual(labels, 
                         expected_labels, 
                         "Incorrect labels ({}), expected {}".format(labels, expected_labels)
                        )
        
    def test_classes(self):
        expected_classes = [0, 1]
        classes = self.test_dga.data_df['class'].unique().tolist()
        self.assertEqual(classes, 
                         expected_classes, 
                         "Incorrect classes ({}), expected {})".format(classes, expected_classes)
                        )
        
    def test_index(self):
        index = self.test_dga.data_df.index.tolist()
        self.assertEqual(len(index), 
                         len(set(index)),
                         "Index contains duplicates ({}), expected {})".format(len(set(index)), len(index)))

    def test_getitem(self):
        item = self.test_dga.__getitem__(0)
        self.assertEqual(len(item), 
                         2,
                         "Get item output is wrong length ({}), expected {})".format(len(item), 2))
    
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 4.424s

OK


In [15]:
# TO:DO Figure out splunk query
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
# def stage(name):
#     with open("data/"+name+".csv", 'r') as f:
#         df = pd.read_csv(f)
#     with open("data/"+name+".json", 'r') as f:
#         param = json.load(f)
#     return df, param

batch_size = 10
data_dir = 'data'
filename = 'alexa_top_1m.csv'
n_records = 100

In [16]:
# # THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df = _get_train_data_loader(batch_size, data_dir, filename, n_records)# print(df.describe())
print(df)
param = {'feature_variables': ['domain'], 'target_variables': ['benign', 'malicious']}
print(param)

Get train data loader.
{'feature_variables': ['domain'], 'target_variables': ['benign', 'malicious']}


In [25]:
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence, pack_padded_sequence
import torch.nn.functional as F

class RNNEstimator(nn.Module):
    """
    RNN Estimator for generating sequences of target variables.
    """
    
    def __init__(self, input_features=65, hidden_dim=12, n_layers=2, output_dim=9, embedding_dim=5, batch_size=10):
        super(RNNEstimator, self).__init__()

        # Variables
        self.hidden_dim = hidden_dim
        self.hidden_layers = n_layers
        self.batch_size = batch_size
        self.embedding_dim = embedding_dim
        
        # Embedding
        self.embedding = nn.Embedding(input_features, embedding_dim)
        
        # RNN Layer
        self.rnn = nn.RNN(embedding_dim, hidden_dim, n_layers, dropout=0.3, batch_first=True)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
#         self.softmax = nn.LogSoftmax(dim=1)
        
    def init_hidden(self):
        """
        Initialize the hidden and cell states of the LSTM with zeros.
        """
        return torch.zeros(self.hidden_layers, self.batch_size, self.hidden_dim)
        
    def forward(self, x, x_lens, hidden_state):
        """
        Perform a forward pass of our model on batch of tracks.
        """
        # x: (batch_size, longest_sequence, embedding) i.e. 10, 32, 5
        # hidden size: (hidden_layers, batch_size, hidden_dim) i.e. 2, 10, 30

        # x_embed: (batch_size, longest_sequence, 1?, embedding_size) i.e. 
        embed_x = self.embedding(x)
        
        x_packed = pack_padded_sequence(embed_x, x_lens, batch_first=True, enforce_sorted=False)

        # Passing in the input and hidden state into the model and obtaining outputs
        output_packed, hidden_state = self.rnn(x_packed, hidden_state)
        
        output_padded, output_lengths = pad_packed_sequence(output_packed, batch_first=True)
        
        output = F.sigmoid(self.fc(output_padded))
        
        return output, hidden_state, output_lengths

## Stage 2 - create and initialize a model

In [26]:
def init(df, param):
    
    X = df.dataset.data_df
    Y = df.dataset.labels
    
    input_size = 66
    hidden_dim = 30
    num_classes = 2
    learning_rate = 0.001
    embedding_dim = 5
    n_layers = 2
    
    mapping = { key: value for value,key in enumerate(np.unique(Y.to_numpy().reshape(-1))) }
    
    # TODO: Change these
    print("FIT build RNN model with input size " + str(df.dataset.data_len))
    print("FIT build model with target classes " + str(num_classes))
    
    model = {
        "input_size": input_size,
        "hidden_dim": hidden_dim,
        "embedding_dim": embedding_dim,
        "num_classes": num_classes,
        "n_layers": n_layers,
        "learning_rate": learning_rate,
        "mapping": mapping,
        "num_epochs": 5,
        "batch_size": 10,
    }
    
    model["train_loader"] = df
    
#     if 'options' in param:
#         if 'params' in param['options']:
#             if 'epochs' in param['options']['params']:
#                 model['num_epochs'] = int(param['options']['params']['epochs'])
#             if 'batch_size' in param['options']['params']:
#                 model['batch_size'] = int(param['options']['params']['batch_size'])

    # RNN Layer
    model['model'] = RNNEstimator(
        input_features=model["input_size"], 
        hidden_dim=model["hidden_dim"], 
        n_layers=model["n_layers"], 
        output_dim=model["num_classes"],
        embedding_dim=model["embedding_dim"], 
        batch_size=model["batch_size"]
    )
    
    # Define loss and optimizer
    model['criterion'] = torch.nn.CrossEntropyLoss(reduction='sum', ignore_index=0)
    
#     optimizer = optim.Adam(model.parameters())
#     loss_fn = torch.nn.BCELoss()
    
    model['optimizer'] = torch.optim.Adam(model['model'].parameters())
    
    return model

In [27]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

FIT build RNN model with input size 200
FIT build model with target classes 2
{'input_size': 66, 'hidden_dim': 30, 'embedding_dim': 5, 'num_classes': 2, 'n_layers': 2, 'learning_rate': 0.001, 'mapping': {0: 0, 1: 1}, 'num_epochs': 5, 'batch_size': 10, 'train_loader': <torch.utils.data.dataloader.DataLoader object at 0x7fd5cc17cf98>, 'model': RNNEstimator(
  (embedding): Embedding(66, 5)
  (rnn): RNN(5, 30, num_layers=2, batch_first=True, dropout=0.3)
  (fc): Linear(in_features=30, out_features=2, bias=True)
), 'criterion': CrossEntropyLoss(), 'optimizer': Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)}


In [28]:
# def make_target(label, label_to_ix):
#     return torch.LongTensor([label_to_ix[label]])

# label_to_ix = {"benign": 0, "dga": 1}

In [29]:
# all_categories = ["benign", "dga"]
# def categoryFromOutput(output):
#     top_n, top_i = output.topk(1)
#     category_i = top_i[0].item()
#     return all_categories[category_i], category_i

## Stage 3 - fit the model

In [173]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model, df, param):
    # model.fit()
    
    returns = {"message": "model trained"}
    
    model['model'].train() # Make sure that the model is in training mode.+

    for epoch in range(1, model['num_epochs'] + 1):

        avg_loss = 0.0
        
        # Iterate over dataset
        for _, (x_padded, y_padded, x_lens, y_lens) in enumerate(model["train_loader"]):
            
            # Clear stored gradient
            model["optimizer"].zero_grad()

            # Initialize hidden state
            hidden_state =  model['model'].init_hidden()

            # Forward pass
            output, hidden_cell, lengths =  model['model'](x_padded, x_lens, hidden_state)
            batch_ce_loss = 0.0
            
            for i in range(output.size(0)):
                ce_loss = model['criterion'](output[i][lengths[i]-1].unsqueeze(0), y_padded[i])
                batch_ce_loss += ce_loss
                
            avg_loss += batch_ce_loss.item()

            # Backward pass
            batch_ce_loss.backward()

            # Update parameters
            model["optimizer"].step()
            
        if epoch % 50 == 0:
            print('Epoch: {}/{}.............'.format(epoch, model['num_epochs']), end=' ')
            print("Loss: {:.4f}".format(avg_loss / len(model["train_loader"])))
        
        avg_loss = 0.0

    # memorize parameters
    returns['model_epochs'] = model['num_epochs']
    returns['model_batch_size'] = model['batch_size']
    returns['model_loss_acc'] = batch_ce_loss.item()
            
    return returns

In [175]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

{'message': 'model trained', 'model_epochs': 5, 'model_batch_size': 10, 'model_loss_acc': 1.2760212421417236}


In [176]:
print(returns['model_loss_acc'])

NameError: name 'returns' is not defined

In [ ]:
testLoader = _get_train_data_loader(10, data_dir, filename, n_records)# print(df.describe())

In [ ]:
classes = ('benign', 'dga')

In [ ]:
# # print images
# imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
domains.shape

In [ ]:
# # Just return an output given a line
# def evaluate(line_tensor):
#     hidden = rnn.initHidden()
    
#     for i in range(line_tensor.size()[0]):
#         output, hidden = rnn(line_tensor[i], hidden)

#     return output

In [121]:
net = RNNEstimator()
net.load_state_dict(rnn, strict=False)
net.eval()

dataiter = iter(testLoader)
domains, labels, xx_lens, _ = dataiter.next()
# print(domains)

print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(len(labels))))

def evaluate(domains):
    with torch.no_grad():
        for data in testloader:
            input = domains
            hidden = model['model'].init_hidden()
            output = model['model'](input, xx_lens, hidden)[0]
            y_hat = output[:, -1, :]
            print(y_hat)
            _, predicted = torch.max(y_hat, 1)
            prediction = [classes[key] for key in predicted.numpy()]
        
evaluate(domains)

GroundTruth:  benign   dga   dga benign benign   dga   dga   dga benign benign
tensor([[-2.8952, -2.8905],
        [-1.3206, -0.1805],
        [-0.1629, -0.0222],
        [-3.0951, -3.0916],
        [-3.2615, -3.2587],
        [-0.7627, -0.6976],
        [-1.4321, -1.3939],
        [-1.4243, -1.3893],
        [-2.2137, -2.1990],
        [-2.8408, -2.8338]])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


## Stage 4 - apply the model

In [178]:
#TO-DO: How does Splunk stage data for apply method. In a CSV?
def apply(model,df,param):
    
    test_df = _get_train_data_loader(batch_size, data_dir, filename, 100)# print(df.describe())
    classes = {v: k for k, v in model['mapping'].items()}
    
    batch_predictions = []
    with torch.no_grad():
        for domains, labels, xx_lens, _ in test_df:
            print(xx_lens)
            print(yy_lens)
            hidden = model['model'].init_hidden()
            output, _ = model['model'](domains, xx_lens, hidden)
            print(output.shape)
            print(output[0])
            y_hat = output[:, -1, :]
            _, predicted = torch.max(y_hat, 1)
            prediction = [classes[key] for key in predicted.numpy()]
            batch_predictions.append(prediction)
            break

    return batch_predictions

In [179]:
# net = RNNEstimator()
# net.load_state_dict(torch.load(PATH))

# load(name)

In [180]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

Get train data loader.
[30, 28, 29, 14, 10, 9, 29, 34, 38, 31, 4, 12, 29, 30, 31, 31, 29, 8, 13, 11, 31, 19, 10, 28, 9, 28, 10, 28, 12, 11, 14, 20]


NameError: name 'yy_lens' is not defined

## Stage 5 - save the model

In [43]:
# save model to name in expected convention "<algo_name>_<model_name>.h5"
def save(model,name):
    torch.save(model, MODEL_DIRECTORY + name + ".pt")
    return model

In [59]:
save(model, 'dga')

{'input_size': 66,
 'hidden_dim': 30,
 'embedding_dim': 5,
 'num_classes': 2,
 'n_layers': 2,
 'learning_rate': 0.001,
 'mapping': {0: 0, 1: 1},
 'num_epochs': 100,
 'batch_size': 32,
 'train_loader': <torch.utils.data.dataloader.DataLoader at 0x7f1cb02a5550>,
 'model': RNNEstimator(
   (embedding): Embedding(66, 5)
   (rnn): RNN(5, 30, num_layers=2, batch_first=True, dropout=0.3)
   (fc): Linear(in_features=30, out_features=2, bias=True)
   (softmax): LogSoftmax()
 ),
 'criterion': NLLLoss(),
 'optimizer': SGD (
 Parameter Group 0
     dampening: 0
     lr: 0.001
     momentum: 0
     nesterov: False
     weight_decay: 0
 )}

In [60]:
rnn = load('dga')

## Stage 6 - load the model

In [44]:
# load model from name in expected convention "<algo_name>_<model_name>.h5"
def load(name):
    model = torch.load(MODEL_DIRECTORY + name + ".pt")
    return model

## Stage 7 - provide a summary of the model

In [ ]:
# return model summary
def summary(model=None):
    returns = {"version": {"pytorch": torch.__version__} }
    if model is not None:
        if 'model' in model:
            returns["summary"] = str(model)
    return returns

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code